# Древнегреческий язык

**Древнегре́ческий язы́к** (самоназв. **др.-греч.** ἡ Ἑλληνικὴ γλῶσσα (γλῶττα)) — язык **индоевропейской семьи**, предок **греческого языка**, распространённый на территории **греческой ойкумены** в эпоху с начала **II тысячелетия до н. э.** до IV века нашей эры. В наши дни используется в церквях и монастырях страны.

Αα — Альфа, Ββ — Бета, Γγ — Гамма, Δδ — Дельта, Εε — Эпсилон, Ζζ — Дзета, Ηη — Эта, Θθ — Тета, Ιι — Йота, Κκ — Каппа, Λλ — Лямбда, Μμ — Мю, Νν — Ню, Ξξ — Кси, Οο — Омикрон, Ππ — Пи, Ρρ — Ро, Σσς — Сигма, Ττ — Тау, Υυ — Ипсилон, Φφ — Фи, Χχ — Хи, Ψψ — Пси, Ωω — Омега.

Древние греки использовали греческий алфавит, произошедший, как считается, от финикийского письма. В классическом виде греческий алфавит, состоявший из 24 букв, сложился к концу V в. до н. э. В древнейших надписях направление письма шло справа налево, затем в течение некоторого времени использовался способ письма под названием бустрофедон (буквально «поворот быка») — направление письма чередовалось от строки к строке. В IV в. до н. э. окончательно утвердилось современное направление — слева направо. В эллинистическую эпоху в надписях стали применяться надстрочные знаки, обозначавшие три типа ударения и два типа придыхания.

## Постановка задачи

Предлагается реализовать алгоритм Витерби для создания PoS-тэггера.

На языке Python/R необходимо разработать PoS-теггер на базе скрытых марковских цепей и алгоритма Витерби.

In [82]:
import conllu  
from io import open
import pickle 
import numpy as np

In [83]:
direct = "./UD_Ancient_Greek/"
train = "grc_proiel-ud-train.conllu"
test  = "grc_proiel-ud-test.conllu"

In [84]:
data_train = conllu.parse_incr(open(direct+train,'r',encoding = "utf-8"))

In [85]:
c3 = {} 
c2 = {} 
c1 = {} 
cT = {} 
for tokenlist in data_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) 
    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [86]:
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

with open('cs_ancgreek.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2015
200
16
9124


In [87]:
with open('cs_ancgreek.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [88]:
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
k = len(tags)

In [89]:
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['PROPN', 'NOUN', 'DET', 'SCONJ', 'CCONJ', 'VERB', 'ADP', 'ADJ', 'PRON', 'ADV', 'AUX', 'NUM', 'INTJ', 'X']


In [90]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    return y[1:]

In [91]:
data_test = conllu.parse_incr(open(direct+test,'r',encoding = "utf-8"))
test_tags = [] 
predict_tags = [] 

for tokenlist in data_test:
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    predict_tags.append(viterbi(sentence))

In [92]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))

Error =  0.04754495375281613
